## Assignment : Segmenting and clustering the neighborhoods in the city of Toronto

We will first import the libraries.

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!pip install bs4
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


Then, we will have access to the html page of Toronto on Wikipedia and try to create a dataframe from it.

In [3]:
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050."
html_data  = requests.get(url).text

#turning our html into Beautiful Soup
soup = BeautifulSoup(html_data,"html5lib")

#let's have a look at the html through a nested structure
#print(soup.prettify())

In [5]:
tables = soup.find_all('table')
for index,table in enumerate(tables):
    if ("wikitable" in str(table)):
        table_index = index
print('The index of the table we are looking for is',table_index)
#print(tables[table_index].prettify())

The index of the table we are looking for is 0


Now, let's create the dataframe from the table.

In [107]:
df = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

for row in tables[table_index].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        postalcode = col[0].text
        borough = col[1].text
        neighborhood = col[2].text
        df = df.append({"PostalCode":postalcode, "Borough":borough, "Neighborhood":neighborhood}, ignore_index=True)      

print(df.shape)
df.head()

(287, 3)


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


Until now, the dataframe has 287 rows. Let's clean it : taking off the \n, dropping the Not assigned etc.

In [108]:
#drop the \n in Neighborhood column
df["Neighborhood"] = df["Neighborhood"].str.replace("\n", "")

In [109]:
#drop the rows where 'Borough' is not assigned
df.drop(df[df["Borough"]=="Not assigned"].index,inplace=True)

In [110]:
df.reset_index(drop=True, inplace=True)
row = df.shape[0]
print(row)
df.head(10)

210


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Etobicoke,Islington Avenue
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


In [111]:
#group neighborhoods by postal code
i = 0
while i < row : #cheking all the indexes
    k=1
    while (i+k < row) & (df['PostalCode'][i] == df['PostalCode'][i+k]) : #comparing the postal code of two cells, if it is similar :
        df['Neighborhood'][i] = df['Neighborhood'][i] + ', ' + df['Neighborhood'][i+k] #adding the neighborhood in the first cell
        df.drop([i+k],inplace=True) #delete the second row
        k = k+1 #increasing k to compare with the cell of the next row
    i = i+k

KeyError: 210

In [115]:
df.reset_index(drop=True, inplace=True)
row = df.shape[0]
print(row)
df.head(10)

103


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [119]:
#replace the Not assigned Neighborhoods by the value in Borough
for i in range (row): #checking all the indexes
    if df['Neighborhood'][i] == 'Not assigned': #to see if there is a 'not assigned' value for neighborhood
        df['Neighborhood'][i] = df['Borough'][i] #and replace it

In [121]:
print('This dataframe has', df.shape[0], 'rows.')

This dataframe has 103 rows.
